In [ ]:
#データの読み込み
import pandas as pd
import numpy as np
import time
import sklearn
from sklearn.metrics import mean_squared_error


csv_df = pd.read_csv('H28.csv',encoding="utf8") 
len(csv_df)
csv_df.info()
csv_df.head()

csv_df2 = pd.read_csv('H29.csv',encoding="utf8") 
len(csv_df2)
csv_df2.info()
csv_df2.head()

csv_df3 = pd.read_csv('H30.csv',encoding="utf8") 
len(csv_df3)
csv_df3.info()
csv_df3.head()

csv_df4 = pd.read_csv('R1.csv',encoding="utf8") 
len(csv_df4)
csv_df4.info()
csv_df4.head()

csv_df5 = pd.read_csv('R2.csv',encoding="utf8") 
len(csv_df5)
csv_df5.info()
csv_df5.head()

In [ ]:
#関数の定義
#データのスケール変換の関数（０〜１のスケールに変換）
def scaler(array):
    
    M = max(array)
    m = min(array)
    if M != m:
        scaled_array = (array - m*np.ones(shape = array.shape))/(M - m)
        def f(x):
            return (x - m)/(M - m)
    else:
        scaled_array = array - m*np.ones(shape = array.shape)
        def f(x):   
            return x - m
#scaled_arrayは変換後のデータ,fは個別に値が与えられた時にscaled_arrayのデータと同様に変換するための関数            
    return scaled_array, f

#データの前処理の関数
def preprocessing(x):
    xl = []
    prpl = []
    for i in range(len(x.T)):
        l = scaler(x.T[i])[0]
        prpl.append(scaler(x.T[i])[1])
        xl.append(l)
    
    return np.array(xl).T, prpl #np.array(xl).Tは前処理後のデータ,prplは各項目のスケールに合わせるための関数を格納した配列

#最も近い値に値を変換する関数
#モデルから出力された値を最も近い実際の地目コードに変換する
def nearvalue(x, l):
    d = []
    for i in range(len(l)):
        d.append(abs(x-l[i]))
        
    return l[d.index(min(d))]

#決定係数を計算する関数
#決定係数を求めるメソッドがないモデルなどの検証に使う
def r_2(v, w):
    
    s1 = np.sum((v-w)**2)
    s2 = np.sum((v-np.mean(v))**2)
    
    return 1-(s1/s2)

#正解率を計算する関数
#地目コードとの一致率を計算するメソッドがないモデルなどの検証に使う
def accuracy(v,w):
    u = v-w
    a = 0
    for d in u:
        if d != 0:
            a+=1
            
    return 1-(a/len(u))

In [ ]:
#データの前処理のプログラム
import numpy as np
import time
import sklearn
ev = csv_df.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev2 = csv_df2.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev3 = csv_df3.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev4 = csv_df4.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev5 = csv_df5.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])

x_H28 = np.array(ev.values)
y_H28 = np.array(csv_df['現況地目コード'].values)
x_H29 = np.array(ev2.values)
y_H29 = np.array(csv_df2['現況地目コード'].values)
x_H30 = np.array(ev3.values)
y_H30 = np.array(csv_df3['現況地目コード'].values)
x_R1 = np.array(ev4.values)
y_R1 = np.array(csv_df4['現況地目コード'].values)
x_R2 = np.array(ev5.values)
y_R2 = np.array(csv_df5['現況地目コード'].values)

x =np.concatenate([x_H28, x_H29])
x =np.concatenate([x, x_H30])
x =np.concatenate([x, x_R1])
x =np.concatenate([x, x_R2])
y =np.concatenate([y_H28, y_H29])
y =np.concatenate([y, y_H30])
y =np.concatenate([y, y_R1])
y =np.concatenate([y, y_R2])
y_class = list(set(y))

prpx = preprocessing(x)[1]
x = preprocessing(x)[0]
x_H28 = x[0:len(x_H28)]
x_H29 = x[len(x_H28):len(x_H28)+len(x_H29)]
x_H30 = x[len(x_H28)+len(x_H29):len(x_H28)+len(x_H29)+len(x_H30)]
x_R1 = x[len(x_H28)+len(x_H29)+len(x_H30):len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1)]
x_R2 = x[len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1):]

In [ ]:
from sklearn import neural_network
from sklearn.metrics import mean_squared_error
import sklearn

x1 = x_H28[np.argsort(x_H28[:,0])]
x2 = x_H29[np.argsort(x_H29[:,0])]
x3 = x_H30[np.argsort(x_H30[:,0])]
x4 = x_R1[np.argsort(x_R1[:,0])]
x_train = np.concatenate([x1,x2])
x_train = np.concatenate([x_train,x3])

y1 = y_H29[np.argsort(x_H29[:,0])][:len(x1)]
y2 = y_H30[np.argsort(x_H30[:,0])][:len(x2)]
y3 = y_R1[np.argsort(x_H30[:,0])][:len(x3)]
y_train = np.concatenate([y1,y2])
y_train = np.concatenate([y_train,y3])

clf = neural_network.MLPRegressor(activation="logistic", alpha=0.0001)
clf.fit(x_train, y_train)

def f(n):
    
    n = prpx[0](n)

    i = np.where(x_R1.T[0]==n)
    x_test = [x_R1[i[0][0]]]
    i = np.where(x_R2.T[0]==n)
    y_test = [y_R2[i[0][0]]]
 
 #    y_tr_predict = np.array([nearvalue(i, y_class) for i in clf.predict(x_train)])
    y_predict = np.array([nearvalue(i, y_class) for i in clf.predict(x_test)])
    
    return y_test, y_predict

In [ ]:
pl = []
al = []
for n in np.array(ev4.values).T[0]:
    al.append(f(n)[0][0])
    pl.append(f(n)[1][0])
    
pl = np.array(pl)
al = np.array(al)
print(f'決定係数{r_2(al,pl)}')
print(f'正解率{accuracy(pl,al)}')